In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [ ]:
heatmap = np.load('../data/activity/heatmap_dailymail_ordered_max.npy')

In [ ]:
scaled_heatmap = preprocessing.scale(heatmap)

In [ ]:
from annoy import AnnoyIndex

In [ ]:
scaled_heatmap.shape

In [ ]:
t = AnnoyIndex(scaled_heatmap.shape[1], metric='euclidean')

In [ ]:
heatmap[heatmap > 0] = 1
for i, user in enumerate(heatmap):
    t.add_item(i, user)

In [ ]:
t.build(10)

In [ ]:
items = t.get_nns_by_item(0, scaled_heatmap.shape[0])

In [ ]:
ordered_heatmap = np.zeros(scaled_heatmap.shape)

In [ ]:
for index, element in enumerate(items):
    ordered_heatmap[index] = heatmap[element]

In [ ]:
ordered_heatmap[ordered_heatmap > 0 ] = 1

In [ ]:
df = pd.read_csv('../data/activity/daily-mail.csv')

In [ ]:
df['min_timestamp'] = pd.to_datetime(df['min_timestamp'])
df['max_timestamp'] = pd.to_datetime(df['max_timestamp'])

In [ ]:
df['min_week_start'] = df['min_timestamp'].dt.to_period('W').apply(lambda r: r.start_time)
df['max_week_start'] = df['max_timestamp'].dt.to_period('W').apply(lambda r: r.start_time)

In [ ]:
from datetime import timedelta, date

def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n)

start_dt = df['min_timestamp'].min()
end_dt = df['max_timestamp'].max()
dates = []
for dt in daterange(start_dt, end_dt):
    dates.append(dt)

In [ ]:
dates_df = pd.DataFrame(dates)
dates_df = pd.to_datetime(dates_df[0])
dates_df = dates_df.reset_index()

In [ ]:
dates_df['week_of_date'] = dates_df[0].dt.to_period('W').apply(lambda r: r.start_time)

In [ ]:
unqiue_dates = pd.to_datetime(dates_df['week_of_date']).unique()
unqiue_dates.sort()

In [ ]:
def save_to_pdf(filename, ax):
    fig = ax.get_figure()
    fig.savefig(filename, bbox_inches='tight')

In [ ]:
import matplotlib.dates as mdates
import matplotlib.pyplot as plt

myFmt = mdates.DateFormatter('%Y')

fig, ax = plt.subplots(figsize=(13,6))
# ax.xaxis.set_major_formatter(FuncFormatter(format_fn))
ax.imshow(ordered_heatmap, interpolation="nearest", cmap='Blues', aspect='auto', vmax= 1, vmin= 0)
index_to_date = pd.DataFrame(unqiue_dates).to_dict()[0]
ax.set_xticklabels([index_to_date[date].year for date in [0, 0, 100, 200, 300, 400, 500, 600]])
ax.set(xlabel='Time', ylabel='Users ', title='Guardian')
save_to_pdf('./plots/guardian-user_activity_over_weeks.pdf', ax)